# Hacer

- Separar día/mes de fecha
- Codificar variables categóricas

### Definiciones

- Encoding de etiquetas: Día de la semana, se puede ordenar
- Binary encoding: Resto

## Usar

- Regresión lineal
- Árbol de decisión

https://chatgpt.com/share/671b97fb-7904-8006-853c-7d28072ea3c5

- Algún método no supervisado?

In [99]:
# Se importan librerías
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error # ver, porque is deprecated
import category_encoders as ce

In [100]:
# Carga del dataset
dataset = pd.read_csv("clean_data.csv")

# Se eliminan las filas que contienen "Otros" en la columna "tipo_cobro"
# Se toma esta decisión porque su impacto es mínimo
dataset = dataset[dataset["tipo_cobro"] != "Otros"]

# Se separa el día y el mes del año en columnas separadas
# Necesitamos que las variables sean numéricas
dataset["nro_dia"] = pd.to_datetime(dataset["fecha"]).dt.day
dataset["nro_mes"] = pd.to_datetime(dataset["fecha"]).dt.month
dataset["anio"] = pd.to_datetime(dataset["fecha"]).dt.year
dataset.drop("fecha", axis=1, inplace=True)

# Se codifica la categoría día de la semana con label encoding (codificación de etiquetas)
# Se toma esta decisión porque la variable es ordinal
day_encoding = {
    'Lunes': 0,
    'Martes': 1,
    'Miercoles': 2,
    'Jueves': 3,
    'Viernes': 4,
    'Sabado': 5,
    'Domingo': 6
}

dataset['dia_semana'] = dataset['dia'].map(day_encoding)
dataset.drop("dia", axis=1, inplace=True)

# Se codifica el resto de las variables categóricas con binary encoding (codificación binaria)
binary_encoder = ce.BinaryEncoder(cols=["id_peaje", "sentido", "tipo_vehiculo", "tipo_cobro"])
dataset = binary_encoder.fit_transform(dataset)

In [101]:
# Se muestra un fragmento del dataset resultante
dataset.sample(10)

,hora,id_peaje_0,id_peaje_1,id_peaje_2,sentido_0,sentido_1,tipo_vehiculo_0,tipo_vehiculo_1,tipo_cobro_0,tipo_cobro_1,tipo_cobro_2,tipo_cobro_3,pasos,nro_dia,nro_mes,anio,dia_semana
4334769,1.0,1,0,0,1,0,0,1,0,1,1,0,9,5,9,2023,1
629290,15.0,1,0,0,1,0,0,1,0,0,1,0,2744,6,12,2016,1
1440836,9.0,0,1,0,1,0,0,1,0,1,1,1,7,26,2,2018,0
3965359,18.0,0,1,1,0,1,0,1,0,0,0,1,30,24,12,2022,5
1384546,0.0,1,0,1,1,0,0,1,0,1,0,0,22,26,1,2018,4
1058059,10.0,0,0,1,1,0,0,1,0,0,0,1,3,31,7,2017,0
2490656,4.0,1,0,0,0,1,0,1,0,1,1,1,6,26,8,2019,0
815725,2.0,0,1,0,1,0,1,0,0,1,0,0,4,19,3,2017,6
3152944,7.0,0,1,0,1,0,1,0,0,0,1,0,9,19,8,2021,3
1409245,22.0,1,0,0,0,1,0,1,0,1,0,0,541,9,2,2018,4


In [102]:
# Eliminamos las columnas sentido_1 y tipo_vehiculo_1 porque no aportan información
dataset.drop(["sentido_1", "tipo_vehiculo_1"], axis=1, inplace=True)

In [103]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4473491 entries, 0 to 4473490
Data columns (total 15 columns):
 #   Column           Dtype  
---  ------           -----  
 0   hora             float64
 1   id_peaje_0       int64  
 2   id_peaje_1       int64  
 3   id_peaje_2       int64  
 4   sentido_0        int64  
 5   tipo_vehiculo_0  int64  
 6   tipo_cobro_0     int64  
 7   tipo_cobro_1     int64  
 8   tipo_cobro_2     int64  
 9   tipo_cobro_3     int64  
 10  pasos            int64  
 11  nro_dia          int32  
 12  nro_mes          int32  
 13  anio             int32  
 14  dia_semana       int64  
dtypes: float64(1), int32(3), int64(11)
memory usage: 460.8 MB


In [104]:
# Cambiarle el nombre a las columnas sentido y tipo_vehículo para saber qué significa cada valor

# Regresión lineal

In [105]:
# Dividir en variables predictoras (X) y variable objetivo (y)
# Ver la razón por la que dejando 'pasos' se reduce mucho el error
x = dataset[['hora', 'id_peaje_0', 'id_peaje_1', 'id_peaje_2', 'sentido_0', 'tipo_vehiculo_0', 'tipo_cobro_0', 'tipo_cobro_1', 'tipo_cobro_2', 'tipo_cobro_3','pasos', 'nro_dia', 'nro_mes', 'anio', 'dia_semana']]  
y = dataset['pasos'] 

# Dividir en conjunto de entrenamiento y de prueba
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=50)

# Instanciar el modelo
linear_model = LinearRegression()

# Entrenar el modelo
linear_model.fit(x_train, y_train)

# Predecir sobre el conjunto de prueba
y_pred = linear_model.predict(x_test)

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio de la regresión lineal múltiple:", mse)


Error cuadrático medio de la regresión lineal múltiple: 4.389362474120446e-25


In [106]:
# Función para procesar los dataset
def procesar_dataset(filepath):
    # Carga del dataset
    dataset = pd.read_csv(filepath)

    # Se eliminan las filas que contienen "Otros" en la columna "tipo_cobro"
    # Se toma esta decisión porque su impacto es mínimo
    dataset = dataset[dataset["tipo_cobro"] != "Otros"]

    # Se separa el día y el mes del año en columnas separadas
    # Necesitamos que las variables sean numéricas
    dataset["nro_dia"] = pd.to_datetime(dataset["fecha"]).dt.day
    dataset["nro_mes"] = pd.to_datetime(dataset["fecha"]).dt.month
    dataset["anio"] = pd.to_datetime(dataset["fecha"]).dt.year
    dataset.drop("fecha", axis=1, inplace=True)

    # Se codifica la categoría día de la semana con label encoding (codificación de etiquetas)
    # Se toma esta decisión porque la variable es ordinal
    day_encoding = {
        'Lunes': 0,
        'Martes': 1,
        'Miercoles': 2,
        'Jueves': 3,
        'Viernes': 4,
        'Sabado': 5,
        'Domingo': 6
    }

    dataset['dia_semana'] = dataset['dia'].map(day_encoding)
    dataset.drop("dia", axis=1, inplace=True)

    # Se codifica el resto de las variables categóricas con binary encoding (codificación binaria)
    binary_encoder = ce.BinaryEncoder(cols=["id_peaje", "sentido", "tipo_vehiculo", "tipo_cobro"])
    dataset = binary_encoder.fit_transform(dataset)
    
    return dataset

In [107]:
# Cargar los datos desde los archivos
dataset_train = procesar_dataset("clean_data.csv")
dataset_test = procesar_dataset("2024_data.csv")

# Separar las variables predictoras (X) y la variable objetivo (y) para el conjunto de entrenamiento
x_train = dataset_train[['hora', 'id_peaje_0', 'id_peaje_1', 'id_peaje_2', 'sentido_0', 'tipo_vehiculo_0', 
                         'tipo_cobro_0', 'tipo_cobro_1', 'tipo_cobro_2', 'tipo_cobro_3',  'pasos',
                         'nro_dia', 'nro_mes', 'anio', 'dia_semana']]  
y_train = dataset_train['pasos']

# Separar las variables predictoras (X) y la variable objetivo (y) para el conjunto de prueba
x_test = dataset_test[['hora', 'id_peaje_0', 'id_peaje_1', 'id_peaje_2', 'sentido_0', 'tipo_vehiculo_0',
                       'tipo_cobro_0', 'tipo_cobro_1', 'tipo_cobro_2', 'tipo_cobro_3', 'pasos',
                       'nro_dia', 'nro_mes', 'anio', 'dia_semana']]  
y_test = dataset_test['pasos']
x_test.dropna(inplace=True)
y_test = y_test[x_test.index]
linear_model = LinearRegression()

# Entrenar el modelo
linear_model.fit(x_train, y_train)

# Predecir sobre el conjunto de prueba
y_pred = linear_model.predict(x_test)

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio de la regresión lineal múltiple:", mse)

C:\Users\Guadalupe\AppData\Local\Temp\ipykernel_20008\3687605668.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test.dropna(inplace=True)


Error cuadrático medio de la regresión lineal múltiple: 1.5310130130981274e-22
